In [1]:
import faro
import os


In [2]:
DATASET_URL = "http://groups.csail.mit.edu/vision/LabelMe/NewImages/indoorCVPR_09.tar"

DATASET_TAR = os.path.join(faro.DEFAULT_STORAGE_DIR,'train','indoorCVPR_09.tar')

DATASET_DIR = os.path.join(faro.DEFAULT_STORAGE_DIR,'train','Images')




In [3]:
if not os.path.exists(os.path.join(faro.DEFAULT_STORAGE_DIR,'train')):
    os.mkdir(os.path.join(faro.DEFAULT_STORAGE_DIR,'train'))

In [4]:
# Download the dataset if it does not exist.
if not os.path.exists(DATASET_TAR):
    import urllib
    opener = urllib.request.URLopener()
    print("Downloading dataset (2.4GB) to %s.  This may take some time."%(DATASET_TAR,))
    opener.retrieve(DATASET_URL, DATASET_TAR)
    print("Download complete.")


In [5]:
# Extract the dataset
if not os.path.exists(DATASET_DIR):
    print("Extracting the dataset.")
    ! tar xf $DATASET_TAR
    ! mv Images $DATASET_DIR
    print("Extraction complete.")

In [6]:
!ls $DATASET_DIR

airport_inside	computerroom	     inside_subway   pantry
artstudio	concert_hall	     jewelleryshop   poolinside
auditorium	corridor	     kindergarden    prisoncell
bakery		deli		     kitchen	     restaurant
bar		dentaloffice	     laboratorywet   restaurant_kitchen
bathroom	dining_room	     laundromat      shoeshop
bedroom		elevator	     library	     stairscase
bookstore	fastfood_restaurant  livingroom      studiomusic
bowling		florist		     lobby	     subway
buffet		gameroom	     locker_room     toystore
casino		garage		     mall	     trainstation
children_room	greenhouse	     meeting_room    tv_studio
church_inside	grocerystore	     movietheater    videostore
classroom	gym		     museum	     waitingroom
cloister	hairsalon	     nursery	     warehouse
closet		hospitalroom	     office	     winecellar
clothingstore	inside_bus	     operating_room


In [7]:
import tensorflow.keras as keras


In [8]:
generator = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                         rotation_range=20,
                                                         width_shift_range=0.2,
                                                         height_shift_range=0.2,
                                                         zoom_range=0.2,
                                                        horizontal_flip=True)
my_iter = generator.flow_from_directory(DATASET_DIR,batch_size=64)

Found 15620 images belonging to 67 classes.


In [9]:
tmp = my_iter.next()


In [10]:
tmp[0].shape


(64, 256, 256, 3)

In [11]:
tmp[1]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [12]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

# Generate dummy data
#x_train = np.random.random((100, 100, 100, 3))
#y_train = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)
#x_test = np.random.random((20, 100, 100, 3))
#y_test = keras.utils.to_categorical(np.random.randint(10, size=(20, 1)), num_classes=10)

model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(67, activation='softmax'))

#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(loss='categorical_crossentropy', optimizer=sgd)

#model.fit(x_train, y_train, batch_size=32, epochs=10)
#score = model.evaluate(x_test, y_test, batch_size=32)

Using TensorFlow backend.


In [13]:
from keras.optimizers import SGD,Adam

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
adam = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=sgd)


In [14]:

model.fit_generator(my_iter,steps_per_epoch=128,epochs=25,use_multiprocessing=True)

Epoch 1/25
128/128 [==============================] - 345s 3s/step - loss: 4.0463
Epoch 2/25
128/128 [==============================] - 335s 3s/step - loss: 3.9485
Epoch 3/25
128/128 [==============================] - 334s 3s/step - loss: 3.9058
Epoch 4/25
128/128 [==============================] - 334s 3s/step - loss: 3.8259
Epoch 5/25
128/128 [==============================] - 336s 3s/step - loss: 3.7699
Epoch 6/25
128/128 [==============================] - 331s 3s/step - loss: 3.7600
Epoch 7/25
128/128 [==============================] - 335s 3s/step - loss: 3.7405
Epoch 8/25
128/128 [==============================] - 327s 3s/step - loss: 3.6371
Epoch 9/25
128/128 [==============================] - 337s 3s/step - loss: 3.6041
Epoch 10/25
128/128 [==============================] - 329s 3s/step - loss: 3.5258
Epoch 11/25
128/128 [==============================] - 337s 3s/step - loss: 3.5249
Epoch 12/25
128/128 [==============================] - 326s 3s/step - loss: 3.4811
Epoch 13/25
1